# Enhanced Ship Detection with ESRGAN and YOLOv5

This notebook demonstrates the usage of ESRGAN for image super-resolution, followed by ship detection using YOLOv5. The notebook includes data preprocessing, model usage, and visualization of results.

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import matplotlib.pyplot as plt

# Preprocessing function for images
def preprocess_image(input_image_path):
    """Preprocesses the image and makes it ready for the ESRGAN model."""
    original_image = tf.image.decode_image(tf.io.read_file(input_image_path))
    # Remove the alpha channel if present
    if original_image.shape[-1] == 4:
        original_image = original_image[..., :-1]
    # Ensure the image dimensions are multiples of 4 (requirement of ESRGAN)
    image_dimensions = (tf.convert_to_tensor(original_image.shape[:-1]) // 4) * 4
    cropped_image = tf.image.crop_to_bounding_box(original_image, 0, 0, image_dimensions[0], image_dimensions[1])
    float_image = tf.cast(cropped_image, tf.float32) / 255.0  # Normalize
    batched_image = tf.expand_dims(float_image, 0)
    return batched_image

In [ ]:
# Load the ESRGAN model
ESRGAN_model_path = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
ESRGAN = hub.load(ESRGAN_model_path)

# Example of super-resolution using ESRGAN
image_path = "train_v2/00003e153.jpg"
lr_image = preprocess_image(image_path)

# Perform super-resolution
sr_image = ESRGAN(lr_image)
sr_image = tf.squeeze(sr_image)

# Visualization of the result
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Low Resolution Image')
plt.imshow(lr_image[0])
plt.subplot(1, 2, 2)
plt.title('Super Resolution Image')
plt.imshow(sr_image)
plt.show()

In [ ]:
# PSNR calculation for quality assessment
def calculate_psnr(original, enhanced):
    """Calculates PSNR to evaluate image quality."""
    original = tf.image.convert_image_dtype(original, tf.float32)
    enhanced = tf.image.convert_image_dtype(enhanced, tf.float32)
    return tf.image.psnr(original, enhanced, max_val=1.0)

# Calculate and display PSNR
psnr_value = calculate_psnr(lr_image, tf.expand_dims(sr_image, 0))
print(f"PSNR of the super-resolved image: {psnr_value.numpy()[0]:.2f} dB")